In [1]:
%reset -f
import torch
from torchvision import transforms
from matplotlib import pyplot as plt
from models import TCNmodel
import numpy as np
from random import randrange
import os

train_sample,frame = torch.load('dataset_train.pt')
print(frame)

print(train_sample.shape)

#data_loader = torch.utils.data.DataLoader(train_sample, batch_size = 1, shuffle = True)
# for videos in data_loader:
#     print(videos.size())


tensor([41, 50, 42, 35, 31, 40, 37, 40, 33, 42, 30])
torch.Size([1, 421, 3, 360, 640])


In [2]:
# print(posindex)
# print(negindex)

In [3]:
def DatasetCon(data,frame):
    # construct a F x 3 x C x W x H tensoR
    # 0: anchor, 1: positive, 2: negative
    N,F,C,W,H = data.shape # F: frame
    anchor = data
    idx = np.arange(F)
    count=0

    # positve & negative
    posindex = np.array([])
    negindex = np.array([])
    for j in range(len(frame)):
        f = frame[j]
        Posidx_j = np.zeros(f)
        Negidx_j = np.zeros(f)
        for i in range(f):
        # pp positve
            pp= randrange(-1,1)
            if pp ==-1:
                Posidx_j[i]=-1
            else:
                Posidx_j[i]=1
        
        # pn negative      
            pn = randrange(0,f)
            while pn<=i+2 & pn>=i-2:
                pn=randrange(0,f)
            Negidx_j[i]=pn+count
        
        count = count+f
         
        # fixed index value at 0 frame and the end frame
        Posidx_j[0]=1
        Posidx_j[-1]=-1
        posindex=np.concatenate((posindex,Posidx_j))
        negindex = np.concatenate((negindex,Negidx_j))
    
    posindex = idx+posindex
    positive = data[:,posindex,:,:,:]

    negative = data[:,negindex,:,:,:]
    dataset = torch.cat([anchor,positive,negative])
    dataset = dataset.transpose(0,1)
    return dataset

In [4]:
# dataset = DatasetCon(train_sample,frame)
# print(dataset.shape)
# # data_loader = torch.utils.data.DataLoader(dataset, batch_size = 4, shuffle = True)
# # for videos in data_loader:
# #     print(videos.size())
# #     break


In [5]:
# model=TCNmodel()

In [6]:
# with torch.no_grad():
#     output = model(torch.reshape(videos,[-1,3,360,640]))
# # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# print(output.shape)

In [7]:
# output=output.reshape([4,3,-1])
# print(output.shape)
# anchor = output[:,0,:]
# positive = output[:,1,:]
# negative = output[:,2,:]
# triplet_loss = torch.nn.TripletMarginLoss(margin=0.2, p=2)
# # corr=torch.mm(output,output.transpose(0,1))
# loss = triplet_loss(anchor, positive, negative)
# print(loss)


In [8]:
def train(train_loader, net, optimizer, criterion,device):
    """
    Trains network for one epoch in batches.

    Args:
        train_loader: Data loader for training set.
        net: Neural network model.
        optimizer: Optimizer (e.g. SGD).
        criterion: Loss function (e.g. cross-entropy loss).
    """
  
    avg_loss = 0
    #correct = 0
    total = 0

    # iterate through batches
    for i, data in enumerate(train_loader):
        N,P,C,W,H = data.shape
        # get the inputs; data is a list of [inputs, labels]
        inputs = data.reshape([-1,C,W,H])
        inputs= inputs.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        outputs = outputs.reshape([N,P,-1])
        
        anchor = outputs[:,0,:]
        positive = outputs[:,1,:]
        negative = outputs[:,2,:]
        
        loss = criterion(anchor,positive,negative)
        loss.backward()
        optimizer.step()

        # keep track of loss and accuracy
        avg_loss += loss
        #print(avg_loss)
    return avg_loss

In [9]:
os.makedirs("./checkpoints/", exist_ok=True)
from tqdm import tqdm
epochs = 20

# Create instance of Network
net = TCNmodel()

# Create loss function and optimizer
criterion = torch.nn.TripletMarginLoss(margin=0.2, p=2,reduction='sum')
optimizer = torch.optim.SGD(net.parameters(), lr=1e-3,momentum=0.9)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)

# load pretrained data
check = 590 # enter the latest epoch number you had !! need to enter manually else start with -1
PATH = './checkpoints/{}_{}.pth'.format('SaveModel',check)
net.load_state_dict(torch.load(PATH)['net'])
optimizer.load_state_dict(torch.load(PATH)['optimizer'])
loss = 2
# print(device)
for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    # randomly generate pair dataset
    dataset = DatasetCon(train_sample,frame)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size = 4, shuffle = True)
    # Train on data
    train_loss = train(data_loader,net,optimizer,criterion,device)
    print(train_loss/421)
    if (loss>train_loss.item()/421) | (((epoch+1) % 20 == 0) and epoch):
        torch.save({'loss':train_loss,
            'net': net.state_dict(),
            'optimizer':optimizer.state_dict()
            }, './checkpoints/{}_{}.pth'.format('SaveModel', epoch+1+check))
        loss = train_loss.item()/421


Using cache found in C:\Users\Think/.cache\torch\hub\pytorch_vision_v0.10.0
  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

tensor(1.2814, device='cuda:0', grad_fn=<DivBackward0>)


 10%|████████▎                                                                          | 2/20 [01:48<16:25, 54.75s/it]

tensor(1.5393, device='cuda:0', grad_fn=<DivBackward0>)


 15%|████████████▍                                                                      | 3/20 [02:40<15:18, 54.04s/it]

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [15:44<01:44, 52.28s/it]

tensor(1.8493, device='cuda:0', grad_fn=<DivBackward0>)


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [16:36<00:52, 52.22s/it]

tensor(1.1955, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.4498, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [17:29<00:00, 52.48s/it]


In [10]:
print((3>train_loss.item()))

False


In [11]:
train_loss.item()

610.3861694335938